In [14]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

## Finding composition with Enrichment and Burnup
Testing time for various methods

In [15]:
enr_br = np.array([[3, 40000]])

## Pickle and model

In [16]:
# load pickled dictionary with
# key: isotope
# value: trained model
dep_dict = pickle.load(open('lin_dep.pkl',"rb"))

In [17]:
# get time for rom depletion for one assembly
# given enrichment and burnup value,
# predict UNF composition
%time

def get_a(f):
    z = ''
    for i in f:
        if i.isdigit():
            z += i
    return int(z)

depleted_composition_rom = {}

isos = [x for x in dep_dict.keys() if 'alg' not in x]

for iso in isos:
    if any(i.isdigit() for i in dep_dict[iso + '_alg']):
        deg = get_a(dep_dict[iso + '_alg'])
        poly = sklearn.preprocessing.PolynomialFeatures(degree=deg)
        depleted_composition_rom[iso] = dep_dict[iso].predict(poly.fit_transform(enr_br))[0]
    else:
        depleted_composition_rom[iso] = dep_dict[iso].predict(enr_br)[0]

print('\n')
depleted_composition_rom



CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.29 µs




{'zr-95': 1.1484601702345132e-06,
 'zr-93': 0.0009430319628774451,
 'ra-226': 8.128166743868014e-14,
 'ru-103': 7.082124825317936e-08,
 'ru-106': 9.224997287038025e-05,
 'sb-124': 1.0174619907388335e-09,
 'sb-125': 1.0266742304882404e-05,
 'pu-238': 0.000312965376732103,
 'pu-239': 0.00739254899490264,
 'np-237': 0.0005838254812632688,
 'nb-95': 1.35320028310323e-06,
 'pa-231': 6.353148416196634e-10,
 'pd-107': 0.00033052533120332857,
 'pm-147': 0.0001357246480633197,
 'pu-240': 0.0029328336587442377,
 'pu-241': 0.0018049883785639684,
 'pu-244': 7.250805995100164e-08,
 'pu-242': 0.0007905655177541726,
 'i-129': 0.00020664612676049988,
 'h-3': 6.660353045641339e-08,
 'kr-85': 2.7943487738310585e-05,
 'nb-94': 1.923656976006179e-09,
 'cm-248': 1.14698832624395e-09,
 'cs-134': 0.00012172554020570855,
 'cs-135': 0.0005546829719198416,
 'cm-245': 7.561112809466447e-06,
 'cm-244': 9.684066079661215e-05,
 'cm-246': 8.1009529359867e-07,
 'cm-247': 1.368632630368019e-08,
 'eu-152': 1.8738705313

## Looking up from db

In [18]:

df = pd.read_csv('./curated.csv', index_col=0)

In [19]:
%time
# get time for `look-up' method for one assembly
# find closest assembly to this burnup and enrichment
df['err'] = abs(df['bu'] - enr_br[0][1]) + abs(df['init_enr'] - enr_br[0][0]) * 10000
minimum_err = df['err'].argmin()
df = df.drop(columns='err')
average_assem = df.loc[minimum_err]
print('\n')
depleted_composition_lookup = average_assem.iloc[5:].to_dict()
print(depleted_composition_lookup)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


{'zr-95': 8.419315164246238e-07, 'zr-93': 0.0009609453127874797, 'ra-226': 1.5648069648472898e-13, 'ru-103': 5.234720110751048e-08, 'ru-106': 6.133016596891877e-05, 'sb-124': 1.1752803572644362e-09, 'sb-125': 8.052678395268684e-06, 'pu-238': 0.00044532550982216717, 'pu-239': 0.010483041417679714, 'np-237': 0.0006387829924524144, 'nb-95': 9.929045343839941e-07, 'pa-231': 9.226876169647597e-10, 'pd-107': 0.0003449923747650625, 'pm-147': 9.583790461631682e-05, 'pu-240': 0.0032450850918009947, 'pu-241': 0.0018377200037567013, 'pu-244': 1.1613758098716321e-07, 'pu-242': 0.0006827076275994006, 'i-129': 0.00021439897680375879, 'h-3': 6.417506985546068e-09, 'kr-85': 2.442767857833397e-05, 'nb-94': 2.0475333937856238e-09, 'cm-248': 1.4156604316092271e-08, 'cs-134': 0.00010709894511074417, 'cs-135': 0.0007524624352977171, 'cm-245': 1.9413393608700712e-05, 'cm-244': 0.00015320506773508578, 'cm-246': 3.314676400249005e-06, 'cm-247':

/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  """
